In [1]:
from pygraylog.graylogapi import GraylogAPI
import json
import sys
import numpy as np
import pandas as pd
import datetime

In [2]:
log_level = 0

graylog_con={
        "user":"pavel",
        "password":"O777ver7k1",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [3]:
#format like 2017-03-06T16:28
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')
#query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'
query='protocol:ss7 AND (tcap_otid:"a2:2a:0c:40" OR tcap_dtid:"a2:2a:0c:40")'

gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo)

In [4]:
def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

def LoadGraylogResult(exportsearch_results):
        jsonResult = json.loads(exportsearch_results)
        outputResult = list()
        i=0
        if "total_results" in jsonResult:
            outLog("Graylog search found {} messages".format(jsonResult["total_results"]),0)

        # Get the results and store them
            for result in jsonResult["messages"]:  
                if "message" in  result and isinstance(result["message"], dict):
                    outputResult.append(result["message"])
                    i+=1
                    if i%100==0: outLog("loaded {} records from Graylog".format(i),0)
            outLog("Graylog search load {} messages".format(len(outputResult)),0)
        else:
            outLog("Graylog search error: {}".format(exportsearch_results),0)
        return outputResult
    
data=LoadGraylogResult(gres)

Graylog search found 59 messages
Graylog search load 59 messages


In [5]:
#Загружаем данные в df

columns=list(data[0].keys())
values = list(data[0].values())
arr_len = len(values)
df=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
for i in range(len(data)-1):
    columns=list(data[i+1].keys())
    values = list(data[i+1].values())
    arr_len = len(values)
    df1=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
    df=pd.concat([df, df1],axis=0,join='outer')
df=df.reset_index(drop=True)
df['tcap_otid']=df['tcap_otid'].fillna(0)
df['tcap_dtid']=df['tcap_dtid'].fillna(0)

df[[
    'm3ua_OPC','m3ua_DPC',
    'sccp_calling_nai','sccp_calling_digits','sccp_calling_ri','sccp_calling_ssn',
    'sccp_called_nai','sccp_called_digits','sccp_called_ri','sccp_called_ssn',
    'sccp_class','sccp_message_type','protocol_name','protocol','protocol_version','opCode_localValue','TCAP_type',
    'tcap_otid','tcap_dtid',
    'e164_msisdn','e212_imsi']].head(3)

,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,sccp_called_ssn,...,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,781,800,4,97254120634,0,6,4,919434399505,0,7,...,0x00000009,map,ss7,2,56,End,0,a2:2a:0c:40,NaN,NaN
1,801,782,4,919434399505,0,7,4,972549612539521,0,6,...,0x00000009,map,ss7,NaN,56,Begin,a2:2a:0c:40,0,NaN,425019612539521
2,1962,2506,4,46731726312,0,8,4,46731726312,0,8,...,0x0000000a,SCCP,ss7,NaN,44,Begin,a2:2a:0c:40,0,46731726312,240400000038251


In [6]:
#Функция для сборки транзакции

def xdr_compile(otid,ind,df_xdr,df):
    dtid=111
    df_xdr_temp=df_xdr
    for i in range(ind-1,-1,-1):
        if df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['tcap_otid']==0\
        and (df.iloc[i]['TCAP_type']=='End' or df.iloc[i]['TCAP_type']=='Abort'):
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
            return df_xdr_temp
        elif df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['TCAP_type']=='Continue' and df.iloc[i]['tcap_otid']<>dtid:
            dtid=df.iloc[i]['tcap_otid']
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
        elif df.iloc[i]['TCAP_type']=='Continue'\
        and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
        elif (df.iloc[i]['TCAP_type']=='End' or df.iloc[i]['TCAP_type']=='Abort')\
        and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
            return df_xdr_temp
    return df_xdr_temp

In [7]:
#Сборка датафрейма XDR

columns=list(df.keys())
df_xdr_res=pd.DataFrame(columns=columns)
for i in range(len(df)-1,-1,-1):
    #if df.iloc[i]['TCAP_type']=='Begin' and (df.iloc[i]['e212_imsi']=='425019612512015'):
    if df.iloc[i]['TCAP_type']=='Begin':
        values = list(df.iloc[i].values)
        arr_len = len(values)
        df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
        #df.drop([i],inplace=True)
        df_xdr_res=pd.concat([df_xdr_res,xdr_compile(df.iloc[i]['tcap_otid'],i,df_temp,df)],axis=0,join='outer')

df_xdr_res=df_xdr_res.reset_index(drop=True)

df_xdr_res[['timestamp',
    'm3ua_OPC','m3ua_DPC',
    'sccp_calling_nai','sccp_calling_digits','sccp_calling_ri','sccp_calling_ssn',
    'sccp_called_nai','sccp_called_digits','sccp_called_ri','sccp_called_ssn',
    'sccp_class','sccp_message_type','protocol_name','protocol','protocol_version','opCode_localValue','TCAP_type',
    'tcap_otid','tcap_dtid',
    'e164_msisdn','e212_imsi']]

,timestamp,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,...,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,2018-01-12T04:18:05.193Z,801,782,4,79037023999,0,7,4,972549612533571,0,...,0x00000009,map,ss7,3,56,Begin,a2:2a:0c:40,0,NaN,425019612533571
1,2018-01-12T04:18:05.226Z,781,800,4,97254120634,0,6,4,79037023999,0,...,0x00000009,map,ss7,NaN,56,End,0,a2:2a:0c:40,NaN,NaN
2,2018-01-12T04:28:28.156Z,701,7138,4,972559900040,0,7,4,972559912011881,0,...,0x00000009,map,ss7,NaN,56,Begin,a2:2a:0c:40,0,NaN,425191201188170
3,2018-01-12T04:28:28.254Z,7138,701,4,972559900050,0,6,4,972559900040,0,...,0x00000009,map,ss7,NaN,56,End,0,a2:2a:0c:40,NaN,NaN
4,2018-01-12T04:29:28.859Z,2506,1962,4,97254120661,0,6,4,491722211494,0,...,0x00000009,map,ss7,3,3,Begin,a2:2a:0c:40,0,NaN,425019610602426
5,2018-01-12T04:29:29.644Z,1962,2506,4,491722211494,0,149,4,97254120661,0,...,0x00000009,map,ss7,3,NaN,End,0,a2:2a:0c:40,NaN,NaN
6,2018-01-12T05:12:59.942Z,801,782,4,79037013999,0,7,4,972549612054504,0,...,0x00000009,map,ss7,NaN,56,Begin,a2:2a:0c:40,0,NaN,425019612054504
7,2018-01-12T05:12:59.978Z,782,800,4,97254120634,0,6,4,79037013999,0,...,0x00000009,map,ss7,NaN,56,End,0,a2:2a:0c:40,NaN,NaN
8,2018-01-12T06:02:15.583Z,801,782,4,8613741022,0,149,4,972549612507523,0,...,0x00000009,map,ss7,NaN,56,Begin,a2:2a:0c:40,0,NaN,425019612507523
9,2018-01-12T06:02:15.599Z,781,800,4,97254120634,0,6,4,8613741022,0,...,0x00000009,map,ss7,NaN,56,End,0,a2:2a:0c:40,NaN,NaN
